# 14. Xử lý Ngôn ngữ Tự nhiên: Tiền Huấn luyện
Con người luôn có nhu cầu được giao tiếp. Chính từ nhu cầu cơ bản này mà một lượng lớn dữ liệu văn bản được tạo ra mỗi ngày. Với lượng dữ liệu văn bản đa dạng từ mạng xã hội, ứng dụng trò chuyện, email, đánh giá sản phẩm, tài liệu nghiên cứu và sách báo, việc giúp máy tính hiểu được những dữ liệu này trở nên quan trọng, nhằm đưa ra cách thức hỗ trợ hoặc quyết định dựa trên ngôn ngữ của con người.

Xử lý ngôn ngữ tự nhiên nghiên cứu sự tương tác bằng ngôn ngữ tự nhiên giữa máy tính và con người. Trong thực tế, việc sử dụng các kỹ thuật xử lý ngôn ngữ tự nhiên để xử lý và phân tích dữ liệu văn bản (ngôn ngữ tự nhiên của con người) rất phổ biến, chẳng hạn như các mô hình ngôn ngữ trong Section 8.3 hay các mô hình dịch máy trong Section 9.5.

Để hiểu dữ liệu văn bản, ta có thể bắt đầu với cách biểu diễn loại dữ liệu này, chẳng hạn xem mỗi từ hay từ con như một token riêng lẻ. Trong chương này, biểu diễn của mỗi token có thể được tiền huấn luyện trên một kho ngữ liệu lớn, sử dụng các mô hình word2vec, GloVe, hay embedding cho từ con. Sau khi tiền huấn luyện, biểu diễn của mỗi token có thể là một vector. Tuy nhiên, biểu diễn này vẫn không đổi dù ngữ cảnh xung quanh bất kể là gì. Ví dụ, biểu diễn vector của từ “bank” là giống nhau trong câu “go to the bank to deposit some money” (ra ngân hàng để gửi tiền) và “go to the bank to sit down” (ra bờ hồ ngồi hóng mát). Do đó, nhiều mô hình tiền huấn luyện gần đây điều chỉnh biểu diễn của cùng một token với các ngữ cảnh khác nhau. Trong số đó có BERT, một mô hình sâu hơn rất nhiều dựa trên bộ mã hóa Transformer. Trong chương này, ta sẽ tập trung vào cách tiền huấn luyện các biểu diễn như vậy cho văn bản, như được mô tả trong Fig. 14.1.

![](images/nlp-map-pretrain.svg)

Fig. 14.1 Các biểu diễn văn bản được tiền huấn luyện có thể được truyền vào các kiến trúc học sâu khác nhau cho các ứng dụng xử lý ngôn ngữ tự nhiên xuôi dòng khác nhau. Chương này tập trung vào cách tiền huấn luyện biểu diễn văn bản ngược dòng (upstream).

Như mô tả trong Fig. 14.1, các biểu diễn văn bản được tiền huấn luyện có thể được truyền vào những kiến trúc học sâu cho các ứng dụng xử lý ngôn ngữ tự nhiên xuôi dòng khác nhau. Chúng tôi sẽ trình bày các phần này trong Section 15.

## 14.1. Embedding Từ (word2vec)
Ngôn ngữ tự nhiên là một hệ thống phức tạp mà con người sử dụng để diễn đạt ngữ nghĩa. Trong hệ thống này, từ là đơn vị cơ bản của ngữ nghĩa. Như tên gọi của nó, một vector từ (**word vector**) là một vector được sử dụng để biểu diễn một từ. Vector từ cũng có thể được xem là vector đặc trưng của một từ. Kỹ thuật ánh xạ từ ngữ sang vector số thực còn được gọi là kỹ thuật embedding từ (**word embedding**). Trong vài năm gần đây, embedding từ dần trở thành kiến thức cơ bản trong xử lý ngôn ngữ tự nhiên.

### 14.1.1. Tại sao không Sử dụng Vector One-hot?
Chúng ta đã sử dụng vector one-hot để đại diện cho từ (thực chất là ký tự) trong Section 8.5. Nhớ lại rằng khi giả sử số lượng các từ riêng biệt trong từ điển (tức kích thước từ điển) là  $N$ , mỗi từ có thể tương ứng một-một với các số nguyên liên tiếp từ $0$ đến  $N−1$ , được gọi là chỉ số của từ. Giả sử chỉ số của một từ là  $i$ . Để thu được biểu diễn vector one-hot của từ đó, ta tạo một vector có  $N$  phần tử có giá trị là 0 và đặt phần tử thứ  $i$  bằng $1$. Theo đó, mỗi từ được biểu diễn dưới dạng vector có độ dài  $N$  có thể được trực tiếp đưa vào mạng nơ-ron.

Mặc dù rất dễ xây dựng các vector one-hot, nhưng chúng thường không phải là lựa chọn tốt. Một trong những lý do chính là các vector one-hot không thể biểu diễn một cách chính xác độ tương tự giữa các từ khác nhau, chẳng hạn như độ tương tự cô-sin mà ta thường sử dụng. Độ tương tự cô-sin của hai vectors  $\mathbf{x}, \mathbf{y} \in \mathbb{R}^d$  là giá trị cô-sin của góc giữa chúng:

<center>$\frac{\mathbf{x}^\top \mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|} \in [-1, 1]$</center>

Do độ tương tự cô-sin giữa các vector one-hot của bất kỳ hai từ khác nhau nào đều bằng 0, nên rất khó sử dụng vector one-hot để biểu diễn độ tương tự giữa các từ khác nhau.

[Word2vec](https://code.google.com/archive/p/word2vec/) là một công cụ được phát minh để giải quyết vấn đề trên. Nó biểu diễn mỗi từ bằng một vector có độ dài cố định và sử dụng những vector này để biểu thị tốt hơn độ tương tự và và các quan hệ loại suy (**analogy relationship**) giữa các từ. Công cụ Word2vec gồm hai mô hình: skip-gam [Mikolov et al., 2013b] và túi từ liên tục (**continuous bag of words – CBOW**) [Mikolov et al., 2013a]. Tiếp theo, ta sẽ xem xét hai mô hình này và phương pháp huấn luyện chúng.

### 14.1.2. Mô hình Skip-Gram
Mô hình skip-gam giả định rằng một từ có thể được sử dụng để sinh ra các từ xung quanh nó trong một chuỗi văn bản. Ví dụ, giả sử chuỗi văn bản là “the”, “man”, “loves”, “his” và “son”. Ta sử dụng “loves” làm từ đích trung tâm và đặt kích thước cửa sổ ngữ cảnh bằng 2. Như mô tả trong Fig. 14.1.1, với từ đích trung tâm “loves”, mô hình skip-gram quan tâm đến xác suất có điều kiện sinh ra các từ ngữ cảnh (“the”, “man”, “his” và “son”) nằm trong khoảng cách không quá 2 từ:

$P("the","man","his","son"∣"loves")$
 
Ta giả định rằng, với từ đích trung tâm cho trước, các từ ngữ cảnh được sinh ra độc lập với nhau. Trong trường hợp này, công thức trên có thể được viết lại thành

$P("the"∣"loves")⋅P("man"∣"loves")⋅P("his"∣"loves")⋅P("son"∣"loves")$

![](images/skip-gram.svg)

Fig. 14.1.1 Mô hình skip-gram quan tâm đến xác suất có điều kiện sinh ra các từ ngữ cảnh với một từ đích trung tâm cho trước.

Trong mô hình skip-gam, mỗi từ được biểu diễn bằng hai vector  $d$ -chiều để tính xác suất có điều kiện. Giả sử chỉ số của một từ trong từ điển là  $i$ , vector của từ được biểu diễn là  $\mathbf{v}_i\in\mathbb{R}^d$  khi từ này là từ đích trung tâm và là  $\mathbf{u}_i\in\mathbb{R}^d$  khi từ này là một từ ngữ cảnh. Gọi  $c$  và  $o$  lần lượt là chỉ số của từ đích trung tâm  $w_c$  và từ ngữ cảnh  $w_o$  trong từ điển. Có thể thu được xác suất có điều kiện sinh ra từ ngữ cảnh cho một từ đích trung tâm cho trước bằng phép toán softmax trên tích vô hướng của vector:

<center>$P(w_o \mid w_c) = \frac{\text{exp}(\mathbf{u}_o^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)}$</center>
 
trong đó, tập chỉ số trong bộ từ vựng là  V={0,1,…,|V|−1} . Giả sử trong một chuỗi văn bản có độ dài  T , từ tại bước thời gian  t  được ký hiệu là  w(t) . Giả sử rằng các từ ngữ cảnh được sinh độc lập với từ trung tâm cho trước. Khi kích thước cửa sổ ngữ cảnh là  m , hàm hợp lý (**likelihood**) của mô hình skip-gam là xác suất kết hợp sinh ra tất cả các từ ngữ cảnh với bất kỳ từ trung tâm cho trước nào

<center>$\prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)})$</center>
 
Ở đây, bất kỳ bước thời gian nào nhỏ hơn 1 hoặc lớn hơn  T  đều có thể được bỏ qua.

### 14.1.3. Huấn luyện Mô hình Skip-Gram
Các tham số trong mô hình skip-gram là vector từ đích trung tâm và vector từ ngữ cảnh cho từng từ riêng lẻ. Trong quá trình huấn luyện, chúng ta sẽ học các tham số mô hình bằng cách cực đại hóa hàm hợp lý, còn gọi là ước lượng hợp lý cực đại. Việc này tương tự với việc giảm thiểu hàm mất mát sau đây:

<center>$- \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \text{log}\, P(w^{(t+j)} \mid w^{(t)})$</center>
 
Nếu ta dùng SGD, thì trong mỗi vòng lặp, ta chọn ra một chuỗi con nhỏ hơn bằng việc lấy mẫu ngẫu nhiên để tính toán mất mát cho chuỗi con đó, rồi sau đó tính gradient để cập nhật các tham số mô hình. Điểm then chốt của việc tính toán gradient là tính gradient của logarit xác suất có điều kiện cho vector từ trung tâm và vector từ ngữ cảnh. Đầu tiên, theo định nghĩa ta có

<center>$\log P(w_o \mid w_c) =
\mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)\right)$</center>
 
Thông qua phép tính đạo hàm, ta nhận được giá trị gradient  vc  từ công thức trên.

\begin{split}\begin{aligned}
\frac{\partial \text{log}\, P(w_o \mid w_c)}{\partial \mathbf{v}_c}
&= \mathbf{u}_o - \frac{\sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^\top \mathbf{v}_c)\mathbf{u}_j}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)}\\
&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} \left(\frac{\text{exp}(\mathbf{u}_j^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)}\right) \mathbf{u}_j\\
&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.
\end{aligned}\end{split}
 
Phép tính cho ra xác suất có điều kiện cho mọi từ có trong từ điển với từ đích trung tâm  wc  cho trước. Sau đó, ta lại sử dụng phương pháp đó để tìm gradient cho các vector từ khác.

Sau khi huấn luyện xong, với từ bất kỳ có chỉ số là  $i$  trong từ điển, ta sẽ nhận được tập hai vector từ  $v_i$  và  $u_i$ . Trong các ứng dụng xử lý ngôn ngữ tự nhiên, vector từ đích trung tâm trong mô hình skip-gram thường được sử dụng để làm vector biểu diễn một từ.

### 14.1.4. Mô hình Túi từ Liên tục (CBOW)
Mô hình túi từ liên tục (Continuous bag of words - CBOW) tương tự như mô hình skip-gram. Khác biệt lớn nhất là mô hình CBOW giả định rằng từ đích trung tâm được tạo ra dựa trên các từ ngữ cảnh phía trước và sau nó trong một chuỗi văn bản. Với cùng một chuỗi văn bản gồm các từ “the”, “man”, “loves”, “his” và “son”, trong đó “love” là từ đích trung tâm, với kích thước cửa sổ ngữ cảnh bằng 2, mô hình CBOW quan tâm đến xác suất có điều kiện để sinh ra từ đích “love” dựa trên các từ ngữ cảnh “the”, “man”, “his” và “son” (minh họa ở Fig. 14.1.2) như sau:

$P("loves"∣"the","man","his","son")$

![](images/cbow.svg)

Fig. 14.1.2 Mô hình CBOW quan tâm đến xác suất có điều kiện tạo ra từ đích trung tâm dựa trên các từ ngữ cảnh cho trước.

Vì có quá nhiều từ ngữ cảnh trong mô hình CBOW, ta sẽ lấy trung bình các vector từ của chúng và sau đó sử dụng phương pháp tương tự như trong mô hình skip-gram để tính xác suất có điều kiện. Giả sử  vi∈Rd  và  ui∈Rd  là vector từ ngữ cảnh và vector từ đích trung tâm của từ có chỉ số  i  trong từ điển (lưu ý rằng các ký hiệu này ngược với các ký hiệu trong mô hình skip-gram). Gọi  c  là chỉ số của từ đích trung tâm  wc , và  o1,…,o2m  là chỉ số các từ ngữ cảnh  wo1,…,wo2m  trong từ điển. Do đó, xác suất có điều kiện sinh ra từ đích trung tâm dựa vào các từ ngữ cảnh cho trước là

<center>$P(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\text{exp}\left(\frac{1}{2m}\mathbf{u}_c^\top (\mathbf{v}_{o_1} + \ldots, + \mathbf{v}_{o_{2m}}) \right)}{ \sum_{i \in \mathcal{V}} \text{exp}\left(\frac{1}{2m}\mathbf{u}_i^\top (\mathbf{v}_{o_1} + \ldots, + \mathbf{v}_{o_{2m}}) \right)}$</center>

Để rút gọn, ký hiệu  $\mathcal{W}_o= \{w_{o_1}, \ldots, w_{o_{2m}}\}$ , và  $\bar{\mathbf{v}}_o = \left(\mathbf{v}_{o_1} + \ldots, + \mathbf{v}_{o_{2m}} \right)/(2m)$ . Phương trình trên được đơn giản hóa thành

<center>$P(w_c \mid \mathcal{W}_o) = \frac{\exp\left(\mathbf{u}_c^\top \bar{\mathbf{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)}$</center>

Cho một chuỗi văn bản có độ dài  T , ta giả định rằng từ xuất hiện tại bước thời gian  t  là  w(t) , và kích thước của cửa sổ ngữ cảnh là  m . Hàm hợp lý của mô hình CBOW là xác suất sinh ra bất kỳ từ đích trung tâm nào dựa vào những từ ngữ cảnh.

<center>$\prod_{t=1}^{T}  P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)})$</center>

### 14.1.5. Huấn luyện Mô hình CBOW
Quá trình huấn luyện mô hình CBOW khá giống với quá trình huấn luyện mô hình skip-gram. Uớc lượng hợp lý cực đại của mô hình CBOW tương đương với việc cực tiểu hóa hàm mất mát:

<center>$-\sum_{t=1}^T  \text{log}\, P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}).$</center>
 
Lưu ý rằng

<center>$\log\,P(w_c \mid \mathcal{W}_o) = \mathbf{u}_c^\top \bar{\mathbf{v}}_o - \log\,\left(\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)\right)$</center>
 
Thông qua phép đạo hàm, ta có thể tính log của xác suất có điều kiện của gradient của bất kỳ vector từ ngữ cảnh nào  $i = 1, \ldots, 2m$ trong công thức trên.

<center>$\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}} = \frac{1}{2m} \left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} \frac{\exp(\mathbf{u}_j^\top \bar{\mathbf{v}}_o)\mathbf{u}_j}{ \sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \bar{\mathbf{v}}_o)} \right) = \frac{1}{2m}\left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j \right)$</center>
 
Sau đó, ta sử dụng cùng phương pháp đó để tính gradient cho các vector của từ khác. Không giống như mô hình skip-gam, trong mô hình CBOW ta thường sử dụng vector từ ngữ cảnh làm vector biểu diễn một từ.

### 14.1.6. Tóm tắt
- Vector từ là một vector được sử dụng để biểu diễn một từ. Kỹ thuật ánh xạ các từ sang vector số thực còn được gọi là kỹ thuật embedding từ.
- Word2vec bao gồm cả mô hình túi từ liên tục (CBOW) và mô hình skip-gam. Mô hình skip-gam giả định rằng các từ ngữ cảnh được sinh ra dựa trên từ đích trung tâm. Mô hình CBOW giả định rằng từ đích trung tâm được sinh ra dựa trên các từ ngữ cảnh.

### 14.1.7. Bài tập
1. Độ phức tạp tính toán của mỗi gradient là bao nhiêu? Nếu từ điển chứa một lượng lớn các từ, điều này sẽ gây ra vấn đề gì?
2. Có một số cụm từ cố định trong tiếng Anh bao gồm nhiều từ, chẳng hạn như “new york”. Bạn sẽ huấn luyện các vector từ của chúng như thế nào? Gợi ý: Xem phần 4 trong bài báo Word2vec[2].
3. Sử dụng mô hình skip-gam làm ví dụ để tìm hiểu về thiết kế của mô hình word2vec. Mối quan hệ giữa tích vô hướng của hai vector từ và độ tương tự cô-sin trong mô hình skip-gam là gì? Đối với một cặp từ có ngữ nghĩa gần nhau, tại sao hai vector từ này lại thường có độ tương tự cô-sin cao?